Library:

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, TensorDataset, DataLoader, random_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import os

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from torch.cuda.amp import GradScaler, autocast  # for improving training efficiency
import torch.optim.lr_scheduler as lr_scheduler

from google.colab import drive

import subprocess
import sys

try:
  from coral_pytorch.losses import CornLoss
  from coral_pytorch.dataset import corn_label_from_logits
except ImportError:
  subprocess.check_call([sys.executable, "-m", "pip", "install", "coral-pytorch"])
  from coral_pytorch.losses import CornLoss
  from coral_pytorch.dataset import corn_label_from_logits

GPU

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Access Google Drive

In [18]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Prepare Save Dir

In [19]:
model_path = '/content/drive/MyDrive/STAT441_kaggle_model/FFNN'  # Replace with your desired path
if not os.path.exists(model_path):
  os.makedirs(model_path)

Set Seed

In [20]:
torch.manual_seed(441)
torch.cuda.manual_seed(441)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Load Data & Data Processing

In [21]:
train_file_path = '/content/drive/MyDrive/STAT441_kaggle_data/Xy_train.npz'
train_data = np.load(train_file_path, mmap_mode='r')
X_train = train_data['X']
y_train = train_data['y']

test_file_path = '/content/drive/MyDrive/STAT441_kaggle_data/X_test.npz'
test_data = np.load(test_file_path, mmap_mode='r')
X_test = test_data['X']

In [22]:
# Check the Shape of the Arrays
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)

input_size = X_train.shape[1]
print("Input Size:", input_size)

Shape of X_train: (400000, 2134)
Shape of y_train: (400000,)
Shape of X_test: (100000, 2134)
Input Size: 2134


Identify Columns with Zero Variance & Remove Duplicates

In [23]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

zero_variance_columns = X_train_df.var() == 0

X_train_df = X_train_df.loc[:, ~zero_variance_columns]
X_test_df = X_test_df.loc[:, ~zero_variance_columns]

X_train = X_train_df.values
X_test = X_test_df.values

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (400000, 2113)
Shape of y_train: (400000,)
Shape of X_test: (100000, 2113)


##Warning: Very Slow!!!!!!!!!!!!!!!!!!!!##

In [ ]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

X_train_df = X_train_df.T.drop_duplicates().T
X_test_df = X_test_df.T.drop_duplicates().T

X_train = X_train_df.values
X_test = X_test_df.values

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

non_duplicate_columns = X_train_df.columns

# Optionally, save these column names for future use
# For example, you can save them to a file
with open("/content/drive/MyDrive/STAT441_kaggle_data/non_duplicate_columns.txt", 'w') as f:
  for col in non_duplicate_columns:
    f.write("%s\n" % col)

##Do this instead if non_duplicate_columns.txt exists!##

In [24]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

with open("/content/drive/MyDrive/STAT441_kaggle_data/non_duplicate_columns.txt", 'r') as f:
  non_duplicate_cols = [line.strip() for line in f]

# Now, whenever you have a new DataFrame, simply call this function
correct_column_indices = [int(col) for col in non_duplicate_cols]  # Convert to integers if necessary
X_train_df = X_train_df[correct_column_indices]
X_test_df = X_test_df[correct_column_indices]

X_train = X_train_df.values
X_test = X_test_df.values

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

input_size = X_train.shape[1]
print("Input Size:", input_size)

Shape of X_train: (400000, 2096)
Shape of X_test: (100000, 2096)
Input Size: 2096


Prepare Standard Normalization Data

In [25]:
standard_scaler = StandardScaler()
X_train_standard_normalized = standard_scaler.fit_transform(X_train)
X_test_standard_normalized = standard_scaler.transform(X_test)
y_train_tensor = torch.Tensor(y_train)

train_batch_size = 128

X_train_standard_normalized_tensor = torch.Tensor(X_train_standard_normalized)
print('train x standard normalized format: ', X_train_standard_normalized_tensor.shape, X_train_standard_normalized_tensor.dtype)
print('train_y format: ', y_train_tensor.shape, y_train_tensor.dtype)

train_standard_normalized_dataset = TensorDataset(X_train_standard_normalized_tensor, y_train_tensor)
train_standard_normalized_dataloader = DataLoader(train_standard_normalized_dataset, batch_size = train_batch_size, shuffle = True, pin_memory=True)

train x standard normalized format:  torch.Size([400000, 2096]) torch.float32
train_y format:  torch.Size([400000]) torch.float32


In [26]:
X_test_standard_normalized_tensor = torch.Tensor(X_test_standard_normalized)
print('test x standard normalized format: ', X_test_standard_normalized_tensor.shape, X_test_standard_normalized_tensor.dtype)

test_standard_normalized_dataset = TensorDataset(X_test_standard_normalized_tensor)
test_standard_normalized_dataloader = DataLoader(test_standard_normalized_dataset, batch_size = 1)

test x standard normalized format:  torch.Size([100000, 2096]) torch.float32


In [27]:
def add_gaussian_noise(tensor, noise_level=0.1):
    return tensor + torch.randn(tensor.size()) * noise_level

# Apply noise to the training data
X_train_noisy_01 = add_gaussian_noise(X_train_standard_normalized_tensor, noise_level=0.1)
# Create TensorDataset with the noisy data
train_noisy_dataset_01 = TensorDataset(X_train_noisy_01, y_train_tensor)
train_noisy_loader_01 = DataLoader(train_noisy_dataset_01, batch_size=train_batch_size, shuffle=True, pin_memory=True)

# Apply noise to the training data
X_train_noisy_001 = add_gaussian_noise(X_train_standard_normalized_tensor, noise_level=0.01)
# Create TensorDataset with the noisy data
train_noisy_dataset_001 = TensorDataset(X_train_noisy_001, y_train_tensor)
train_noisy_loader_001 = DataLoader(train_noisy_dataset_001, batch_size=train_batch_size, shuffle=True, pin_memory=True)

In [28]:
combined_train_subsampler_01 = ConcatDataset([train_standard_normalized_dataset, train_noisy_dataset_01])
combined_train_loader_01 = DataLoader(combined_train_subsampler_01, batch_size=train_batch_size, shuffle=True, pin_memory=True)

combined_train_subsampler_001 = ConcatDataset([train_standard_normalized_dataset, train_noisy_dataset_001])
combined_train_loader_001 = DataLoader(combined_train_subsampler_001, batch_size=train_batch_size, shuffle=True, pin_memory=True)

Model Definition

In [29]:
class FFNN(nn.Module):
  def __init__(self, input_size, hidden_sizes, output_size, dropout_rates, batch_norm=False):
    super(FFNN, self).__init__()
    self.layers = nn.ModuleList()  # ModuleList to hold all layers
    self.hidden_sizes = hidden_sizes
    self.dropout_rates = dropout_rates

    # Create layers based on the hidden_sizes and dropout_rates
    last_size = input_size
    for hidden_size, dropout_rate in zip(hidden_sizes, dropout_rates):
      self.layers.append(nn.Linear(last_size, hidden_size))
      if batch_norm:
        self.layers.append(nn.BatchNorm1d(hidden_size))
      self.layers.append(nn.Dropout(dropout_rate))
      last_size = hidden_size

    # Output layer
    self.layers.append(nn.Linear(last_size, output_size))

  def forward(self, x):
    for layer in self.layers[:-1]:
      if isinstance(layer, nn.Linear):
        x = F.relu(layer(x))
      else:
        x = layer(x)
    # No activation function in the last layer
    x = self.layers[-1](x)
    return x

  def generate_filename(self):
    filename = "FFNN_" + "_".join(f"{hs}-{dr}" for hs, dr in zip(self.hidden_sizes, self.dropout_rates))
    return filename


##simple##

In [ ]:
def train_model(model, criterion, optimizer, total_epochs):
  # Ensure the model is in training mode
  model.train()

  # Retraining loop
  for epoch in range(total_epochs):
    running_loss = 0.0
    total_mae = 0.0
    total_samples = 0

    for inputs, labels in train_standard_normalized_dataloader:
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs).squeeze()
      loss = criterion(outputs, labels.float())
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      # Compute MAE
      clamped_outputs = torch.clamp(outputs, 0, 7).round()
      total_mae += torch.sum(torch.abs(clamped_outputs - labels.float())).item()
      total_samples += labels.size(0)

    # Average loss and MAE for this epoch
    epoch_loss = running_loss / len(train_standard_normalized_dataloader)
    epoch_mae = total_mae / total_samples

    print(f'Epoch {epoch + 1}/{total_epochs}, Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')

  torch.save(model, os.path.join(model_path, model.generate_filename() + ".pth"))

In [ ]:
def test_and_save_prediction(model):
  # Ensure the model is in evaluation mode
  model.eval()

  # List to store predictions
  predictions = []
  raw_predictions = []

  with torch.no_grad():
    for inputs in test_standard_normalized_dataloader:
      inputs = inputs[0]
      inputs = inputs.to(device)

      # Forward pass
      outputs = model(inputs).squeeze()
      raw_predictions.append(outputs.cpu().item())

      # Process the outputs (e.g., rounding/clamping)
      processed_output = torch.clamp(outputs.round(), 0, 7)
      predictions.append(int(processed_output.cpu().item()))

  # 'predictions' now contains the processed predictions for your test dataset
  ids = range(0, X_test.shape[0])
  elements = {"ID": ids, "Predicted": predictions}
  predict_df = pd.DataFrame(elements)
  predict_df.to_csv(os.path.join(model_path, model.generate_filename() + '.csv'),index=False)

  return raw_predictions

##ordinal regression & noise##

In [30]:
def train_model_ordinal(model, criterion, optimizer, total_epochs, dataloader, msg):
  # Ensure the model is in training mode
  model.train()

  # Retraining loop
  for epoch in range(total_epochs):
    running_loss = 0.0
    total_mae = 0.0
    total_samples = 0

    for inputs, labels in dataloader:
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs).squeeze()
      loss = criterion(outputs, labels.float())
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      # Compute MAE
      clamped_outputs = corn_label_from_logits(outputs)
      total_mae += torch.sum(torch.abs(clamped_outputs - labels.float())).item()
      total_samples += labels.size(0)

    # Average loss and MAE for this epoch
    epoch_loss = running_loss / len(dataloader)
    epoch_mae = total_mae / total_samples

    print(f'Epoch {epoch + 1}/{total_epochs}, Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')

  torch.save(model, os.path.join(model_path, model.generate_filename() + msg +"_ordinal.pth"))

In [31]:
def test_and_save_prediction_ordinal(model, msg):
  # Ensure the model is in evaluation mode
  model.eval()

  # List to store predictions
  predictions = []

  with torch.no_grad():
    for inputs in test_standard_normalized_dataloader:
      inputs = inputs[0]
      inputs = inputs.to(device)

      # Forward pass
      outputs = model(inputs).squeeze()

      # Process the outputs (e.g., rounding/clamping)
      processed_output = corn_label_from_logits(outputs.unsqueeze(0))
      predictions.append(int(processed_output.cpu().item()))

  # 'predictions' now contains the processed predictions for your test dataset
  ids = range(0, X_test.shape[0])
  elements = {"ID": ids, "Predicted": predictions}
  predict_df = pd.DataFrame(elements)
  predict_df.to_csv(os.path.join(model_path, model.generate_filename() + msg + '_ordinal.csv'),index=False)

##Training & Prediction##

##simple model##

In [ ]:
FFNN_model_final_1 = FFNN(input_size, [512, 128], 1, [0.5, 0.3], True)
FFNN_model_final_1.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(FFNN_model_final_1.parameters(), lr=1e-3, weight_decay=1e-5)
train_model(FFNN_model_final_1, criterion, optimizer, 70)
raw_FFNN_model_final_1 = test_and_save_prediction(FFNN_model_final_1)

In [ ]:
FFNN_model_final_2 = FFNN(input_size, [256, 128], 1, [0.5, 0.3], True)
FFNN_model_final_2.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(FFNN_model_final_2.parameters(), lr=0.0001, weight_decay=1e-5)
train_model(FFNN_model_final_2, criterion, optimizer, 100)
raw_FFNN_model_final_2 = test_and_save_prediction(FFNN_model_final_2)

In [ ]:
FFNN_model_final_3 = FFNN(input_size, [128, 64, 64, 32], 1, [0.5, 0.4, 0.4, 0.3], True)
FFNN_model_final_3.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(FFNN_model_final_3.parameters(), lr=0.001, weight_decay=0)
train_model(FFNN_model_final_3, criterion, optimizer, 100)
raw_FFNN_model_final_3 = test_and_save_prediction(FFNN_model_final_3)

## model with regression only##

In [ ]:
FFNN_model_final_ordinal_1 = FFNN(input_size, [512, 128], 7, [0.5, 0.5], True)
FFNN_model_final_ordinal_1.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_1.parameters(), lr=0.0001, weight_decay=1e-5)
train_model_ordinal(FFNN_model_final_ordinal_1, criterion, optimizer, 80, train_standard_normalized_dataloader, "")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_1, "")

In [ ]:
FFNN_model_final_ordinal_2 = FFNN(input_size, [128, 64, 32], 7, [0.5, 0.3, 0.3], True)
FFNN_model_final_ordinal_2.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_2.parameters(), lr=0.0001, weight_decay=1e-5)
train_model_ordinal(FFNN_model_final_ordinal_2, criterion, optimizer, 100, train_standard_normalized_dataloader, "")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_2, "")

In [ ]:
FFNN_model_final_ordinal_3 = FFNN(input_size, [256, 128], 7, [0.5, 0.5], True)
FFNN_model_final_ordinal_3.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_3.parameters(), lr=0.0001, weight_decay=1e-5)
train_model_ordinal(FFNN_model_final_ordinal_3, criterion, optimizer, 60, train_standard_normalized_dataloader, "")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_3, "")

##model with ordinal regression and noise injection##

###both noise and original dataset###

In [ ]:
FFNN_model_final_ordinal_noisy01_1 = FFNN(input_size, [256, 128, 64], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_noisy01_1.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_noisy01_1.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_noisy01_1, criterion, optimizer, 22, combined_train_loader_01, "_e22_noisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_noisy01_1, "_e22_noisy01")

Epoch 1/22, Loss: 0.4259, MAE: 1.3023
Epoch 2/22, Loss: 0.3834, MAE: 1.1057
Epoch 3/22, Loss: 0.3754, MAE: 1.0686
Epoch 4/22, Loss: 0.3706, MAE: 1.0479
Epoch 5/22, Loss: 0.3671, MAE: 1.0328
Epoch 6/22, Loss: 0.3646, MAE: 1.0219
Epoch 7/22, Loss: 0.3628, MAE: 1.0140
Epoch 8/22, Loss: 0.3619, MAE: 1.0106
Epoch 9/22, Loss: 0.3610, MAE: 1.0055
Epoch 10/22, Loss: 0.3601, MAE: 1.0008
Epoch 11/22, Loss: 0.3595, MAE: 0.9990
Epoch 12/22, Loss: 0.3590, MAE: 0.9957
Epoch 13/22, Loss: 0.3582, MAE: 0.9921
Epoch 14/22, Loss: 0.3580, MAE: 0.9916
Epoch 15/22, Loss: 0.3577, MAE: 0.9896
Epoch 16/22, Loss: 0.3575, MAE: 0.9896
Epoch 17/22, Loss: 0.3567, MAE: 0.9857
Epoch 18/22, Loss: 0.3564, MAE: 0.9839
Epoch 19/22, Loss: 0.3562, MAE: 0.9830
Epoch 20/22, Loss: 0.3561, MAE: 0.9834
Epoch 21/22, Loss: 0.3558, MAE: 0.9822
Epoch 22/22, Loss: 0.3560, MAE: 0.9824


In [ ]:
FFNN_model_final_ordinal_noisy01_2 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_noisy01_2.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_noisy01_2.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_noisy01_2, criterion, optimizer, 28, combined_train_loader_01, "_e28_noisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_noisy01_2, "_e28_noisy01")

Epoch 1/28, Loss: 0.4321, MAE: 1.3400
Epoch 2/28, Loss: 0.3866, MAE: 1.1193
Epoch 3/28, Loss: 0.3791, MAE: 1.0850
Epoch 4/28, Loss: 0.3742, MAE: 1.0631
Epoch 5/28, Loss: 0.3710, MAE: 1.0471
Epoch 6/28, Loss: 0.3684, MAE: 1.0360
Epoch 7/28, Loss: 0.3670, MAE: 1.0292
Epoch 8/28, Loss: 0.3658, MAE: 1.0246
Epoch 9/28, Loss: 0.3648, MAE: 1.0193
Epoch 10/28, Loss: 0.3639, MAE: 1.0158
Epoch 11/28, Loss: 0.3631, MAE: 1.0116
Epoch 12/28, Loss: 0.3625, MAE: 1.0093
Epoch 13/28, Loss: 0.3619, MAE: 1.0065
Epoch 14/28, Loss: 0.3617, MAE: 1.0053
Epoch 15/28, Loss: 0.3613, MAE: 1.0041
Epoch 16/28, Loss: 0.3609, MAE: 1.0013
Epoch 17/28, Loss: 0.3605, MAE: 0.9982
Epoch 18/28, Loss: 0.3603, MAE: 0.9982
Epoch 19/28, Loss: 0.3604, MAE: 0.9990
Epoch 20/28, Loss: 0.3600, MAE: 0.9974
Epoch 21/28, Loss: 0.3596, MAE: 0.9954
Epoch 22/28, Loss: 0.3595, MAE: 0.9948
Epoch 23/28, Loss: 0.3593, MAE: 0.9943
Epoch 24/28, Loss: 0.3592, MAE: 0.9941
Epoch 25/28, Loss: 0.3592, MAE: 0.9941
Epoch 26/28, Loss: 0.3590, MAE: 0.

###only noise###

####already trained####

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_1 = FFNN(input_size, [256, 128, 64], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_1.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_1.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_1, criterion, optimizer, 36, train_noisy_loader_01, "_e36_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_1, "_e36_onlyNoisy01")

Epoch 1/36, Loss: 0.4534, MAE: 1.4270
Epoch 2/36, Loss: 0.3978, MAE: 1.1618
Epoch 3/36, Loss: 0.3868, MAE: 1.1227
Epoch 4/36, Loss: 0.3810, MAE: 1.0958
Epoch 5/36, Loss: 0.3774, MAE: 1.0800
Epoch 6/36, Loss: 0.3746, MAE: 1.0669
Epoch 7/36, Loss: 0.3721, MAE: 1.0554
Epoch 8/36, Loss: 0.3700, MAE: 1.0460
Epoch 9/36, Loss: 0.3685, MAE: 1.0393
Epoch 10/36, Loss: 0.3669, MAE: 1.0329
Epoch 11/36, Loss: 0.3658, MAE: 1.0266
Epoch 12/36, Loss: 0.3649, MAE: 1.0237
Epoch 13/36, Loss: 0.3643, MAE: 1.0207
Epoch 14/36, Loss: 0.3636, MAE: 1.0162
Epoch 15/36, Loss: 0.3631, MAE: 1.0149
Epoch 16/36, Loss: 0.3624, MAE: 1.0113
Epoch 17/36, Loss: 0.3619, MAE: 1.0077
Epoch 18/36, Loss: 0.3614, MAE: 1.0068
Epoch 19/36, Loss: 0.3609, MAE: 1.0051
Epoch 20/36, Loss: 0.3606, MAE: 1.0035
Epoch 21/36, Loss: 0.3605, MAE: 1.0021
Epoch 22/36, Loss: 0.3601, MAE: 1.0009
Epoch 23/36, Loss: 0.3601, MAE: 1.0005
Epoch 24/36, Loss: 0.3599, MAE: 1.0004
Epoch 25/36, Loss: 0.3591, MAE: 0.9963
Epoch 26/36, Loss: 0.3592, MAE: 0.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_2 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_2.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_2.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_2, criterion, optimizer, 48, train_noisy_loader_01, "_e48_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_2, "_e48_onlyNoisy01")

Epoch 1/48, Loss: 0.4652, MAE: 1.5264
Epoch 2/48, Loss: 0.4004, MAE: 1.1751
Epoch 3/48, Loss: 0.3897, MAE: 1.1350
Epoch 4/48, Loss: 0.3847, MAE: 1.1121
Epoch 5/48, Loss: 0.3807, MAE: 1.0935
Epoch 6/48, Loss: 0.3780, MAE: 1.0799
Epoch 7/48, Loss: 0.3758, MAE: 1.0716
Epoch 8/48, Loss: 0.3735, MAE: 1.0617
Epoch 9/48, Loss: 0.3720, MAE: 1.0537
Epoch 10/48, Loss: 0.3707, MAE: 1.0468
Epoch 11/48, Loss: 0.3696, MAE: 1.0425
Epoch 12/48, Loss: 0.3684, MAE: 1.0379
Epoch 13/48, Loss: 0.3678, MAE: 1.0349
Epoch 14/48, Loss: 0.3671, MAE: 1.0295
Epoch 15/48, Loss: 0.3665, MAE: 1.0279
Epoch 16/48, Loss: 0.3660, MAE: 1.0254
Epoch 17/48, Loss: 0.3651, MAE: 1.0223
Epoch 18/48, Loss: 0.3650, MAE: 1.0224
Epoch 19/48, Loss: 0.3643, MAE: 1.0183
Epoch 20/48, Loss: 0.3638, MAE: 1.0158
Epoch 21/48, Loss: 0.3639, MAE: 1.0167
Epoch 22/48, Loss: 0.3634, MAE: 1.0138
Epoch 23/48, Loss: 0.3636, MAE: 1.0141
Epoch 24/48, Loss: 0.3628, MAE: 1.0104
Epoch 25/48, Loss: 0.3626, MAE: 1.0109
Epoch 26/48, Loss: 0.3631, MAE: 1.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_3 = FFNN(input_size, [200, 100], 7, [0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_3.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_3.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_3, criterion, optimizer, 30, train_noisy_loader_01, "_e30_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_3, "_e30_onlyNoisy01")

Epoch 1/30, Loss: 0.4404, MAE: 1.3630
Epoch 2/30, Loss: 0.3893, MAE: 1.1326
Epoch 3/30, Loss: 0.3801, MAE: 1.0998
Epoch 4/30, Loss: 0.3752, MAE: 1.0774
Epoch 5/30, Loss: 0.3718, MAE: 1.0612
Epoch 6/30, Loss: 0.3691, MAE: 1.0490
Epoch 7/30, Loss: 0.3670, MAE: 1.0398
Epoch 8/30, Loss: 0.3654, MAE: 1.0321
Epoch 9/30, Loss: 0.3642, MAE: 1.0249
Epoch 10/30, Loss: 0.3627, MAE: 1.0206
Epoch 11/30, Loss: 0.3613, MAE: 1.0138
Epoch 12/30, Loss: 0.3609, MAE: 1.0111
Epoch 13/30, Loss: 0.3596, MAE: 1.0071
Epoch 14/30, Loss: 0.3592, MAE: 1.0035
Epoch 15/30, Loss: 0.3584, MAE: 0.9997
Epoch 16/30, Loss: 0.3578, MAE: 0.9965
Epoch 17/30, Loss: 0.3572, MAE: 0.9933
Epoch 18/30, Loss: 0.3571, MAE: 0.9953
Epoch 19/30, Loss: 0.3566, MAE: 0.9907
Epoch 20/30, Loss: 0.3565, MAE: 0.9919
Epoch 21/30, Loss: 0.3560, MAE: 0.9884
Epoch 22/30, Loss: 0.3559, MAE: 0.9879
Epoch 23/30, Loss: 0.3555, MAE: 0.9855
Epoch 24/30, Loss: 0.3551, MAE: 0.9842
Epoch 25/30, Loss: 0.3547, MAE: 0.9816
Epoch 26/30, Loss: 0.3545, MAE: 0.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_4 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_4.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_4.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_4, criterion, optimizer, 35, train_noisy_loader_001, "_e35_onlyNoisy001")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_4, "_e35_onlyNoisy001")

Epoch 1/35, Loss: 0.4645, MAE: 1.5187
Epoch 2/35, Loss: 0.4008, MAE: 1.1724
Epoch 3/35, Loss: 0.3901, MAE: 1.1351
Epoch 4/35, Loss: 0.3851, MAE: 1.1107
Epoch 5/35, Loss: 0.3813, MAE: 1.0927
Epoch 6/35, Loss: 0.3785, MAE: 1.0811
Epoch 7/35, Loss: 0.3759, MAE: 1.0684
Epoch 8/35, Loss: 0.3740, MAE: 1.0596
Epoch 9/35, Loss: 0.3720, MAE: 1.0515
Epoch 10/35, Loss: 0.3702, MAE: 1.0437
Epoch 11/35, Loss: 0.3692, MAE: 1.0383
Epoch 12/35, Loss: 0.3682, MAE: 1.0339
Epoch 13/35, Loss: 0.3672, MAE: 1.0307
Epoch 14/35, Loss: 0.3663, MAE: 1.0248
Epoch 15/35, Loss: 0.3656, MAE: 1.0219
Epoch 16/35, Loss: 0.3651, MAE: 1.0192
Epoch 17/35, Loss: 0.3647, MAE: 1.0177
Epoch 18/35, Loss: 0.3642, MAE: 1.0148
Epoch 19/35, Loss: 0.3638, MAE: 1.0135
Epoch 20/35, Loss: 0.3636, MAE: 1.0134
Epoch 21/35, Loss: 0.3633, MAE: 1.0132
Epoch 22/35, Loss: 0.3628, MAE: 1.0105
Epoch 23/35, Loss: 0.3627, MAE: 1.0097
Epoch 24/35, Loss: 0.3625, MAE: 1.0087
Epoch 25/35, Loss: 0.3620, MAE: 1.0073
Epoch 26/35, Loss: 0.3617, MAE: 1.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_5 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_5.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_5.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_5, criterion, optimizer, 55, train_noisy_loader_01, "_e55_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_5, "_e55_onlyNoisy01")

Epoch 1/55, Loss: 0.4588, MAE: 1.4869
Epoch 2/55, Loss: 0.3996, MAE: 1.1701
Epoch 3/55, Loss: 0.3893, MAE: 1.1312
Epoch 4/55, Loss: 0.3842, MAE: 1.1091
Epoch 5/55, Loss: 0.3809, MAE: 1.0922
Epoch 6/55, Loss: 0.3781, MAE: 1.0793
Epoch 7/55, Loss: 0.3756, MAE: 1.0686
Epoch 8/55, Loss: 0.3736, MAE: 1.0595
Epoch 9/55, Loss: 0.3720, MAE: 1.0521
Epoch 10/55, Loss: 0.3705, MAE: 1.0456
Epoch 11/55, Loss: 0.3694, MAE: 1.0407
Epoch 12/55, Loss: 0.3683, MAE: 1.0350
Epoch 13/55, Loss: 0.3676, MAE: 1.0326
Epoch 14/55, Loss: 0.3669, MAE: 1.0296
Epoch 15/55, Loss: 0.3663, MAE: 1.0265
Epoch 16/55, Loss: 0.3658, MAE: 1.0239
Epoch 17/55, Loss: 0.3652, MAE: 1.0215
Epoch 18/55, Loss: 0.3649, MAE: 1.0219
Epoch 19/55, Loss: 0.3640, MAE: 1.0170
Epoch 20/55, Loss: 0.3641, MAE: 1.0162
Epoch 21/55, Loss: 0.3636, MAE: 1.0148
Epoch 22/55, Loss: 0.3634, MAE: 1.0137
Epoch 23/55, Loss: 0.3629, MAE: 1.0111
Epoch 24/55, Loss: 0.3626, MAE: 1.0086
Epoch 25/55, Loss: 0.3623, MAE: 1.0084
Epoch 26/55, Loss: 0.3623, MAE: 1.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_6 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_6.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.SGD(FFNN_model_final_ordinal_onlyNoisy01_6.parameters(), lr=0.001, weight_decay=1e-3, momentum=0.95)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_6, criterion, optimizer, 50, train_noisy_loader_01, "_SGD95_e50_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_6, "_SGD95_e50_onlyNoisy01")

Epoch 1/50, Loss: 0.4430, MAE: 1.4057
Epoch 2/50, Loss: 0.4011, MAE: 1.1880
Epoch 3/50, Loss: 0.3939, MAE: 1.1519
Epoch 4/50, Loss: 0.3902, MAE: 1.1326
Epoch 5/50, Loss: 0.3873, MAE: 1.1166
Epoch 6/50, Loss: 0.3846, MAE: 1.1024
Epoch 7/50, Loss: 0.3822, MAE: 1.0912
Epoch 8/50, Loss: 0.3802, MAE: 1.0805
Epoch 9/50, Loss: 0.3779, MAE: 1.0688
Epoch 10/50, Loss: 0.3757, MAE: 1.0594
Epoch 11/50, Loss: 0.3736, MAE: 1.0506
Epoch 12/50, Loss: 0.3712, MAE: 1.0385
Epoch 13/50, Loss: 0.3696, MAE: 1.0328
Epoch 14/50, Loss: 0.3680, MAE: 1.0238
Epoch 15/50, Loss: 0.3662, MAE: 1.0150
Epoch 16/50, Loss: 0.3648, MAE: 1.0105
Epoch 17/50, Loss: 0.3629, MAE: 1.0013
Epoch 18/50, Loss: 0.3614, MAE: 0.9931
Epoch 19/50, Loss: 0.3601, MAE: 0.9886
Epoch 20/50, Loss: 0.3587, MAE: 0.9818
Epoch 21/50, Loss: 0.3575, MAE: 0.9759
Epoch 22/50, Loss: 0.3563, MAE: 0.9731
Epoch 23/50, Loss: 0.3551, MAE: 0.9667
Epoch 24/50, Loss: 0.3541, MAE: 0.9616
Epoch 25/50, Loss: 0.3535, MAE: 0.9595
Epoch 26/50, Loss: 0.3521, MAE: 0.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_6 = FFNN(input_size, [200, 100, 50], 7, [0.5, 0.5, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_6.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.SGD(FFNN_model_final_ordinal_onlyNoisy01_6.parameters(), lr=0.001, weight_decay=1e-3, momentum=0.90)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_6, criterion, optimizer, 70, train_noisy_loader_01, "_SGD90_e70_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_6, "_SGD90_e70_onlyNoisy01")

Epoch 1/70, Loss: 0.4668, MAE: 1.5279
Epoch 2/70, Loss: 0.4113, MAE: 1.2298
Epoch 3/70, Loss: 0.4024, MAE: 1.1900
Epoch 4/70, Loss: 0.3976, MAE: 1.1704
Epoch 5/70, Loss: 0.3941, MAE: 1.1530
Epoch 6/70, Loss: 0.3912, MAE: 1.1385
Epoch 7/70, Loss: 0.3895, MAE: 1.1322
Epoch 8/70, Loss: 0.3874, MAE: 1.1208
Epoch 9/70, Loss: 0.3856, MAE: 1.1103
Epoch 10/70, Loss: 0.3835, MAE: 1.1012
Epoch 11/70, Loss: 0.3815, MAE: 1.0908
Epoch 12/70, Loss: 0.3799, MAE: 1.0852
Epoch 13/70, Loss: 0.3781, MAE: 1.0743
Epoch 14/70, Loss: 0.3766, MAE: 1.0678
Epoch 15/70, Loss: 0.3749, MAE: 1.0594
Epoch 16/70, Loss: 0.3735, MAE: 1.0536
Epoch 17/70, Loss: 0.3719, MAE: 1.0449
Epoch 18/70, Loss: 0.3704, MAE: 1.0379
Epoch 19/70, Loss: 0.3690, MAE: 1.0310
Epoch 20/70, Loss: 0.3676, MAE: 1.0234
Epoch 21/70, Loss: 0.3665, MAE: 1.0184
Epoch 22/70, Loss: 0.3651, MAE: 1.0101
Epoch 23/70, Loss: 0.3639, MAE: 1.0056
Epoch 24/70, Loss: 0.3624, MAE: 0.9991
Epoch 25/70, Loss: 0.3614, MAE: 0.9924
Epoch 26/70, Loss: 0.3601, MAE: 0.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_9 = FFNN(input_size, [200, 100, 50], 7, [0.7, 0.7, 0.7], True)
FFNN_model_final_ordinal_onlyNoisy01_9.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_9.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_9, criterion, optimizer, 150, train_noisy_loader_01, "_e150_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_9, "_e150_onlyNoisy01")

Epoch 1/150, Loss: 0.5101, MAE: 1.7595
Epoch 2/150, Loss: 0.4204, MAE: 1.2774
Epoch 3/150, Loss: 0.4056, MAE: 1.2138
Epoch 4/150, Loss: 0.4009, MAE: 1.1924
Epoch 5/150, Loss: 0.3984, MAE: 1.1758
Epoch 6/150, Loss: 0.3965, MAE: 1.1653
Epoch 7/150, Loss: 0.3960, MAE: 1.1623
Epoch 8/150, Loss: 0.3952, MAE: 1.1586
Epoch 9/150, Loss: 0.3945, MAE: 1.1535
Epoch 10/150, Loss: 0.3940, MAE: 1.1508
Epoch 11/150, Loss: 0.3933, MAE: 1.1478
Epoch 12/150, Loss: 0.3930, MAE: 1.1451
Epoch 13/150, Loss: 0.3925, MAE: 1.1440
Epoch 14/150, Loss: 0.3923, MAE: 1.1441
Epoch 15/150, Loss: 0.3920, MAE: 1.1407
Epoch 16/150, Loss: 0.3914, MAE: 1.1377
Epoch 17/150, Loss: 0.3916, MAE: 1.1376
Epoch 18/150, Loss: 0.3914, MAE: 1.1386
Epoch 19/150, Loss: 0.3912, MAE: 1.1375
Epoch 20/150, Loss: 0.3912, MAE: 1.1377
Epoch 21/150, Loss: 0.3909, MAE: 1.1363
Epoch 22/150, Loss: 0.3905, MAE: 1.1346
Epoch 23/150, Loss: 0.3907, MAE: 1.1356
Epoch 24/150, Loss: 0.3904, MAE: 1.1351
Epoch 25/150, Loss: 0.3904, MAE: 1.1341
Epoch 26/

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_7 = FFNN(input_size, [200, 100, 50], 7, [0.6, 0.6, 0.6], True)
FFNN_model_final_ordinal_onlyNoisy01_7.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_7.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_7, criterion, optimizer, 72, train_noisy_loader_01, "_e72_batch256_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_7, "_e72_batch256_onlyNoisy01")

Epoch 1/72, Loss: 0.5202, MAE: 1.8686
Epoch 2/72, Loss: 0.4249, MAE: 1.2711
Epoch 3/72, Loss: 0.4049, MAE: 1.1948
Epoch 4/72, Loss: 0.3960, MAE: 1.1612
Epoch 5/72, Loss: 0.3904, MAE: 1.1375
Epoch 6/72, Loss: 0.3870, MAE: 1.1213
Epoch 7/72, Loss: 0.3849, MAE: 1.1097
Epoch 8/72, Loss: 0.3828, MAE: 1.1004
Epoch 9/72, Loss: 0.3806, MAE: 1.0873
Epoch 10/72, Loss: 0.3793, MAE: 1.0817
Epoch 11/72, Loss: 0.3778, MAE: 1.0746
Epoch 12/72, Loss: 0.3767, MAE: 1.0687
Epoch 13/72, Loss: 0.3755, MAE: 1.0636
Epoch 14/72, Loss: 0.3742, MAE: 1.0572
Epoch 15/72, Loss: 0.3729, MAE: 1.0517
Epoch 16/72, Loss: 0.3725, MAE: 1.0495
Epoch 17/72, Loss: 0.3719, MAE: 1.0477
Epoch 18/72, Loss: 0.3710, MAE: 1.0440
Epoch 19/72, Loss: 0.3704, MAE: 1.0411
Epoch 20/72, Loss: 0.3698, MAE: 1.0374
Epoch 21/72, Loss: 0.3693, MAE: 1.0361
Epoch 22/72, Loss: 0.3688, MAE: 1.0341
Epoch 23/72, Loss: 0.3684, MAE: 1.0321
Epoch 24/72, Loss: 0.3680, MAE: 1.0303
Epoch 25/72, Loss: 0.3676, MAE: 1.0280
Epoch 26/72, Loss: 0.3676, MAE: 1.

In [ ]:
FFNN_model_final_ordinal_onlyNoisy01_8 = FFNN(input_size, [200, 100, 50], 7, [0.6, 0.6, 0.6], True)
FFNN_model_final_ordinal_onlyNoisy01_8.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_8.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_8, criterion, optimizer, 100, train_noisy_loader_01, "_e100_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_8, "_e100_onlyNoisy01")

Epoch 1/100, Loss: 0.4895, MAE: 1.6696
Epoch 2/100, Loss: 0.4090, MAE: 1.2172
Epoch 3/100, Loss: 0.3969, MAE: 1.1691
Epoch 4/100, Loss: 0.3926, MAE: 1.1465
Epoch 5/100, Loss: 0.3899, MAE: 1.1316
Epoch 6/100, Loss: 0.3881, MAE: 1.1229
Epoch 7/100, Loss: 0.3864, MAE: 1.1137
Epoch 8/100, Loss: 0.3851, MAE: 1.1084
Epoch 9/100, Loss: 0.3839, MAE: 1.1040
Epoch 10/100, Loss: 0.3830, MAE: 1.0987
Epoch 11/100, Loss: 0.3822, MAE: 1.0958
Epoch 12/100, Loss: 0.3812, MAE: 1.0909
Epoch 13/100, Loss: 0.3806, MAE: 1.0878
Epoch 14/100, Loss: 0.3802, MAE: 1.0869
Epoch 15/100, Loss: 0.3796, MAE: 1.0835
Epoch 16/100, Loss: 0.3792, MAE: 1.0829
Epoch 17/100, Loss: 0.3787, MAE: 1.0809
Epoch 18/100, Loss: 0.3787, MAE: 1.0806
Epoch 19/100, Loss: 0.3781, MAE: 1.0783
Epoch 20/100, Loss: 0.3778, MAE: 1.0766
Epoch 21/100, Loss: 0.3775, MAE: 1.0750
Epoch 22/100, Loss: 0.3776, MAE: 1.0766
Epoch 23/100, Loss: 0.3771, MAE: 1.0743
Epoch 24/100, Loss: 0.3770, MAE: 1.0732
Epoch 25/100, Loss: 0.3767, MAE: 1.0722
Epoch 26/

In [17]:
FFNN_model_final_ordinal_onlyNoisy01_10 = FFNN(input_size, [200, 100, 50], 7, [0.7, 0.6, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_10.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_10.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_10, criterion, optimizer, 70, train_noisy_loader_01, "_e70_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_10, "_e70_onlyNoisy01")

Epoch 1/70, Loss: 0.4833, MAE: 1.6449
Epoch 2/70, Loss: 0.4071, MAE: 1.2080
Epoch 3/70, Loss: 0.3973, MAE: 1.1696
Epoch 4/70, Loss: 0.3938, MAE: 1.1520
Epoch 5/70, Loss: 0.3915, MAE: 1.1418
Epoch 6/70, Loss: 0.3902, MAE: 1.1339
Epoch 7/70, Loss: 0.3891, MAE: 1.1294
Epoch 8/70, Loss: 0.3881, MAE: 1.1254
Epoch 9/70, Loss: 0.3871, MAE: 1.1222
Epoch 10/70, Loss: 0.3865, MAE: 1.1186
Epoch 11/70, Loss: 0.3863, MAE: 1.1179
Epoch 12/70, Loss: 0.3853, MAE: 1.1141
Epoch 13/70, Loss: 0.3852, MAE: 1.1128
Epoch 14/70, Loss: 0.3844, MAE: 1.1107
Epoch 15/70, Loss: 0.3842, MAE: 1.1087
Epoch 16/70, Loss: 0.3838, MAE: 1.1081
Epoch 17/70, Loss: 0.3837, MAE: 1.1064
Epoch 18/70, Loss: 0.3837, MAE: 1.1059
Epoch 19/70, Loss: 0.3832, MAE: 1.1053
Epoch 20/70, Loss: 0.3833, MAE: 1.1063
Epoch 21/70, Loss: 0.3830, MAE: 1.1046
Epoch 22/70, Loss: 0.3827, MAE: 1.1011
Epoch 23/70, Loss: 0.3826, MAE: 1.1018
Epoch 24/70, Loss: 0.3824, MAE: 1.1003
Epoch 25/70, Loss: 0.3824, MAE: 1.1007
Epoch 26/70, Loss: 0.3821, MAE: 1.

In [19]:
FFNN_model_final_ordinal_onlyNoisy01_11 = FFNN(input_size, [300, 200, 100], 7, [0.7, 0.6, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_11.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_11.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_11, criterion, optimizer, 40, train_noisy_loader_01, "_e40_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_11, "_e40_onlyNoisy01")

Epoch 1/40, Loss: 0.4605, MAE: 1.4719
Epoch 2/40, Loss: 0.3990, MAE: 1.1701
Epoch 3/40, Loss: 0.3911, MAE: 1.1404
Epoch 4/40, Loss: 0.3870, MAE: 1.1240
Epoch 5/40, Loss: 0.3854, MAE: 1.1157
Epoch 6/40, Loss: 0.3835, MAE: 1.1082
Epoch 7/40, Loss: 0.3826, MAE: 1.1034
Epoch 8/40, Loss: 0.3816, MAE: 1.0995
Epoch 9/40, Loss: 0.3806, MAE: 1.0965
Epoch 10/40, Loss: 0.3802, MAE: 1.0960
Epoch 11/40, Loss: 0.3796, MAE: 1.0913
Epoch 12/40, Loss: 0.3791, MAE: 1.0890
Epoch 13/40, Loss: 0.3790, MAE: 1.0891
Epoch 14/40, Loss: 0.3786, MAE: 1.0887
Epoch 15/40, Loss: 0.3779, MAE: 1.0833
Epoch 16/40, Loss: 0.3778, MAE: 1.0839
Epoch 17/40, Loss: 0.3776, MAE: 1.0837
Epoch 18/40, Loss: 0.3774, MAE: 1.0831
Epoch 19/40, Loss: 0.3772, MAE: 1.0813
Epoch 20/40, Loss: 0.3771, MAE: 1.0806
Epoch 21/40, Loss: 0.3766, MAE: 1.0787
Epoch 22/40, Loss: 0.3765, MAE: 1.0797
Epoch 23/40, Loss: 0.3765, MAE: 1.0789
Epoch 24/40, Loss: 0.3765, MAE: 1.0794
Epoch 25/40, Loss: 0.3763, MAE: 1.0781
Epoch 26/40, Loss: 0.3765, MAE: 1.

In [20]:
FFNN_model_final_ordinal_onlyNoisy01_12 = FFNN(input_size, [300, 200, 100], 7, [0.6, 0.6, 0.5], True)
FFNN_model_final_ordinal_onlyNoisy01_12.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_12.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_12, criterion, optimizer, 40, train_noisy_loader_01, "_e40_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_12, "_e40_onlyNoisy01")

Epoch 1/40, Loss: 0.4556, MAE: 1.4300
Epoch 2/40, Loss: 0.3970, MAE: 1.1597
Epoch 3/40, Loss: 0.3876, MAE: 1.1239
Epoch 4/40, Loss: 0.3831, MAE: 1.1072
Epoch 5/40, Loss: 0.3801, MAE: 1.0929
Epoch 6/40, Loss: 0.3780, MAE: 1.0836
Epoch 7/40, Loss: 0.3762, MAE: 1.0761
Epoch 8/40, Loss: 0.3752, MAE: 1.0716
Epoch 9/40, Loss: 0.3737, MAE: 1.0650
Epoch 10/40, Loss: 0.3732, MAE: 1.0617
Epoch 11/40, Loss: 0.3716, MAE: 1.0559
Epoch 12/40, Loss: 0.3713, MAE: 1.0548
Epoch 13/40, Loss: 0.3709, MAE: 1.0533
Epoch 14/40, Loss: 0.3702, MAE: 1.0503
Epoch 15/40, Loss: 0.3698, MAE: 1.0482
Epoch 16/40, Loss: 0.3690, MAE: 1.0452
Epoch 17/40, Loss: 0.3689, MAE: 1.0448
Epoch 18/40, Loss: 0.3685, MAE: 1.0422
Epoch 19/40, Loss: 0.3685, MAE: 1.0426
Epoch 20/40, Loss: 0.3681, MAE: 1.0400
Epoch 21/40, Loss: 0.3679, MAE: 1.0401
Epoch 22/40, Loss: 0.3676, MAE: 1.0383
Epoch 23/40, Loss: 0.3671, MAE: 1.0352
Epoch 24/40, Loss: 0.3672, MAE: 1.0382
Epoch 25/40, Loss: 0.3669, MAE: 1.0348
Epoch 26/40, Loss: 0.3667, MAE: 1.

In [21]:
FFNN_model_final_ordinal_onlyNoisy01_13 = FFNN(input_size, [300, 200, 100], 7, [0.7, 0.6, 0.6], True)
FFNN_model_final_ordinal_onlyNoisy01_13.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_13.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_13, criterion, optimizer, 40, train_noisy_loader_01, "_e40_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_13, "_e40_onlyNoisy01")

Epoch 1/40, Loss: 0.4638, MAE: 1.4763
Epoch 2/40, Loss: 0.4030, MAE: 1.1895
Epoch 3/40, Loss: 0.3932, MAE: 1.1524
Epoch 4/40, Loss: 0.3892, MAE: 1.1341
Epoch 5/40, Loss: 0.3871, MAE: 1.1234
Epoch 6/40, Loss: 0.3854, MAE: 1.1165
Epoch 7/40, Loss: 0.3843, MAE: 1.1119
Epoch 8/40, Loss: 0.3837, MAE: 1.1090
Epoch 9/40, Loss: 0.3828, MAE: 1.1041
Epoch 10/40, Loss: 0.3821, MAE: 1.1019
Epoch 11/40, Loss: 0.3813, MAE: 1.0985
Epoch 12/40, Loss: 0.3809, MAE: 1.0978
Epoch 13/40, Loss: 0.3804, MAE: 1.0953
Epoch 14/40, Loss: 0.3799, MAE: 1.0935
Epoch 15/40, Loss: 0.3796, MAE: 1.0904
Epoch 16/40, Loss: 0.3797, MAE: 1.0904
Epoch 17/40, Loss: 0.3792, MAE: 1.0892
Epoch 18/40, Loss: 0.3791, MAE: 1.0894
Epoch 19/40, Loss: 0.3789, MAE: 1.0886
Epoch 20/40, Loss: 0.3787, MAE: 1.0872
Epoch 21/40, Loss: 0.3785, MAE: 1.0877
Epoch 22/40, Loss: 0.3787, MAE: 1.0876
Epoch 23/40, Loss: 0.3779, MAE: 1.0851
Epoch 24/40, Loss: 0.3784, MAE: 1.0868
Epoch 25/40, Loss: 0.3778, MAE: 1.0837
Epoch 26/40, Loss: 0.3777, MAE: 1.

In [32]:
FFNN_model_final_ordinal_onlyNoisy01_14 = FFNN(input_size, [512, 128], 7, [0.6, 0.6], True)
FFNN_model_final_ordinal_onlyNoisy01_14.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_14.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_14, criterion, optimizer, 40, train_noisy_loader_01, "_e40_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_14, "_e40_onlyNoisy01")

Epoch 1/40, Loss: 0.4463, MAE: 1.3683
Epoch 2/40, Loss: 0.3929, MAE: 1.1465
Epoch 3/40, Loss: 0.3832, MAE: 1.1105
Epoch 4/40, Loss: 0.3786, MAE: 1.0935
Epoch 5/40, Loss: 0.3763, MAE: 1.0823
Epoch 6/40, Loss: 0.3745, MAE: 1.0744
Epoch 7/40, Loss: 0.3729, MAE: 1.0665
Epoch 8/40, Loss: 0.3718, MAE: 1.0625
Epoch 9/40, Loss: 0.3708, MAE: 1.0588
Epoch 10/40, Loss: 0.3697, MAE: 1.0549
Epoch 11/40, Loss: 0.3692, MAE: 1.0515
Epoch 12/40, Loss: 0.3685, MAE: 1.0486
Epoch 13/40, Loss: 0.3676, MAE: 1.0439
Epoch 14/40, Loss: 0.3672, MAE: 1.0426
Epoch 15/40, Loss: 0.3670, MAE: 1.0417
Epoch 16/40, Loss: 0.3667, MAE: 1.0401
Epoch 17/40, Loss: 0.3660, MAE: 1.0377
Epoch 18/40, Loss: 0.3660, MAE: 1.0367
Epoch 19/40, Loss: 0.3654, MAE: 1.0329
Epoch 20/40, Loss: 0.3654, MAE: 1.0341
Epoch 21/40, Loss: 0.3652, MAE: 1.0317
Epoch 22/40, Loss: 0.3651, MAE: 1.0320
Epoch 23/40, Loss: 0.3649, MAE: 1.0308
Epoch 24/40, Loss: 0.3644, MAE: 1.0303
Epoch 25/40, Loss: 0.3645, MAE: 1.0292
Epoch 26/40, Loss: 0.3644, MAE: 1.

####new model####

In [33]:
FFNN_model_final_ordinal_onlyNoisy01_15 = FFNN(input_size, [256, 128], 7, [0.6, 0.6], True)
FFNN_model_final_ordinal_onlyNoisy01_15.to(device)
criterion = CornLoss(8)
optimizer = torch.optim.Adam(FFNN_model_final_ordinal_onlyNoisy01_15.parameters(), lr=0.0001, weight_decay=1e-3)
train_model_ordinal(FFNN_model_final_ordinal_onlyNoisy01_15, criterion, optimizer, 40, train_noisy_loader_01, "_e40_onlyNoisy01")
test_and_save_prediction_ordinal(FFNN_model_final_ordinal_onlyNoisy01_15, "_e40_onlyNoisy01")

Epoch 1/40, Loss: 0.4488, MAE: 1.3780
Epoch 2/40, Loss: 0.3943, MAE: 1.1520
Epoch 3/40, Loss: 0.3845, MAE: 1.1170
Epoch 4/40, Loss: 0.3800, MAE: 1.0976
Epoch 5/40, Loss: 0.3777, MAE: 1.0878
Epoch 6/40, Loss: 0.3761, MAE: 1.0815
Epoch 7/40, Loss: 0.3743, MAE: 1.0714
Epoch 8/40, Loss: 0.3732, MAE: 1.0677
Epoch 9/40, Loss: 0.3721, MAE: 1.0623
Epoch 10/40, Loss: 0.3708, MAE: 1.0565
Epoch 11/40, Loss: 0.3702, MAE: 1.0546
Epoch 12/40, Loss: 0.3695, MAE: 1.0530
Epoch 13/40, Loss: 0.3687, MAE: 1.0488
Epoch 14/40, Loss: 0.3684, MAE: 1.0466
Epoch 15/40, Loss: 0.3677, MAE: 1.0430
Epoch 16/40, Loss: 0.3674, MAE: 1.0424
Epoch 17/40, Loss: 0.3670, MAE: 1.0402
Epoch 18/40, Loss: 0.3666, MAE: 1.0376
Epoch 19/40, Loss: 0.3665, MAE: 1.0365
Epoch 20/40, Loss: 0.3664, MAE: 1.0375
Epoch 21/40, Loss: 0.3661, MAE: 1.0350
Epoch 22/40, Loss: 0.3660, MAE: 1.0352
Epoch 23/40, Loss: 0.3657, MAE: 1.0335
Epoch 24/40, Loss: 0.3656, MAE: 1.0326
Epoch 25/40, Loss: 0.3653, MAE: 1.0331
Epoch 26/40, Loss: 0.3655, MAE: 1.

##Load model##

In [ ]:
model_load1 = torch.load("FFNN_[128, 64, 32]_1_[0.5, 0.3, 0.3]_True_0001_500.pth")
model_load1.eval()  # Don't forget to call eval() for inference

In [ ]:
# List to store predictions
predictions_load1 = []

with torch.no_grad():
  for inputs in test_standard_normalized_dataloader:
    inputs = inputs[0]
    inputs = inputs.to(device)

    # Forward pass
    outputs_1 = model_load1(inputs).squeeze()

    # Process the outputs (e.g., rounding/clamping)
    processed_output = torch.clamp(outputs_1.round(), 0, 7)

    # Store the predictions
    predictions_load1.append(int(processed_output.cpu().item()))

# 'predictions' now contains the processed predictions for your test dataset

In [ ]:
ids = range(0, X_test.shape[0])
elements = {"ID": ids, "Predicted": predictions_load1}
predict_df = pd.DataFrame(elements)
predict_df.to_csv('submission_FFNN_load1.csv',index=False)